In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
import statsmodels.formula.api as sfa
import statsmodels.api as sma

# VIF 
from statsmodels.stats.outliers_influence import variance_inflation_factor 

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, LeaveOneOut, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, RFECV
# Terminal --> pip install mlxtend

#from mlxtend.feature_selection

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

* Feature engineering
* issing values
* one hot encoding

In [3]:
# combine the dataset
combined = pd.concat([train,test],ignore_index=True)

In [4]:
# checking the missing values
combined.isnull().sum().sort_values(ascending=False)

Cabin          1014
Survived        418
Age             263
Embarked          2
Fare              1
PassengerId       0
Pclass            0
Name              0
Sex               0
SibSp             0
Parch             0
Ticket            0
dtype: int64

In [5]:
combined.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [6]:
combined.columns


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
combined['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [8]:
cabins = ['C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64', 'E24', 'C90', 'C45', 'E8', 'B101', 'D45', 'C46', 'D30',
       'E121', 'D11', 'E77', 'F38', 'B3', 'D6', 'B82 B84', 'D17', 'A36',
       'B102', 'B69', 'E49', 'C47', 'D28', 'E17', 'A24', 'C50', 'B42',
       'C148', 'B45', 'B36', 'A21', 'D34', 'A9', 'C31', 'B61', 'C53',
       'D43', 'C130', 'C132', 'C55 C57', 'C116', 'F', 'A29', 'C6', 'C28',
       'C51', 'C97', 'D22', 'B10', 'E45', 'E52', 'A11', 'B11', 'C80',
       'C89', 'F E46', 'B26', 'F E57', 'A18', 'E60', 'E39 E41',
       'B52 B54 B56', 'C39', 'B24', 'D40', 'D38', 'C105']

In [9]:
def cabin_labels(x):
    if x in cabins:
        return('cabin_avail')
    else:
        return('missing')

In [10]:
combined.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [11]:
# Lets apply this on cabin column
combined['cabin_cat'] = combined['Cabin'].apply(cabin_labels)

In [12]:
pd.crosstab(combined.Survived, combined.cabin_cat)

cabin_cat,cabin_avail,missing
Survived,,
0.0,68,481
1.0,136,206


## Name

In [13]:
combined.Name[899].split(', ')[1].split('. ')[0]

'Mrs'

In [14]:
combined.Name[899].split(', ')[1].split('. ')[0]

'Mrs'

In [15]:
titless = []
for i in combined.Name:
    titless.append(i.split(', ')[1].split('. ')[0])

In [16]:
combined['Titles']=pd.Series(titless)

In [17]:
combined.Titles.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [18]:
titles_ignore = ['Don', 'Rev', 'Dr', 'Mme',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona']

def notitle(x):
    if x in titles_ignore:
        return('others')
    else:
        return(x)

In [19]:
combined['Titles'] = combined['Titles'].apply(notitle)

In [20]:
combined.drop(['PassengerId','Name','Cabin','Ticket'],axis = 1, inplace=True)

### Family

In [21]:
combined['Family'] = combined.SibSp+combined.Parch+1

In [22]:
def parivar(x):
    if x == 1:
        return('solo')
    elif x ==2:
        return('duo')
    elif x<=4:
        return('small')
    else:
        return('big')

In [23]:
combined['family_cat'] = combined.Family.apply(parivar)

In [24]:
pd.crosstab(combined.family_cat, combined.Survived)

Survived,0.0,1.0
family_cat,,
big,52,10
duo,72,89
small,51,80
solo,374,163


In [28]:
combined.groupby(by = 'Titles')['Age'].describe().T

Titles,Master,Miss,Mr,Mrs,Ms,others
count,53.000000,210.000000,581.000000,170.000000,1.0,31.000000
mean,5.482642,21.774238,32.252151,36.994118,28.0,43.129032
std,4.161554,12.249077,12.422089,12.901767,NaN,12.309189
min,0.330000,0.170000,11.000000,14.000000,28.0,23.000000
25%,2.000000,15.000000,23.000000,27.000000,28.0,32.500000
50%,4.000000,22.000000,29.000000,35.500000,28.0,45.000000
75%,9.000000,30.000000,39.000000,46.500000,28.0,52.500000
max,14.500000,63.000000,80.000000,76.000000,28.0,70.000000


### Missing values

In [33]:
# treating the missing value in columns Age
missing_titles = combined.loc[combined.Age.isnull()]['Titles'].unique()

In [41]:
for i in missing_titles:
    combined.loc[combined.Age.isnull(), 'Age'] = combined.loc[combined.Titles==i,'Age'].median()

In [51]:
# Treating the missing value in column Embarked
combined.loc[combined.Embarked.isnull(),'Embarked'] = combined.Embarked.mode()[0]

In [56]:
# Missing the null values in Fare
combined.loc[combined.Fare.isnull(),'Fare'] = combined.Fare.median()

In [58]:
# Split the data back in train and test
newtrain = combined.loc[0:train.shape[0]-1, ]
newtest = combined.loc[train.shape[0]:, ]

newtrain.shape, newtest.shape

((891, 12), (418, 12))

In [61]:
# lets split the data in x and y
X = newtrain.drop(['Survived'],axis = 1)
y = newtrain.Survived.astype(int)

newtest.drop(['Survived'],axis = 1, inplace = True)

### Model building

* The first model in classicfication which is alo known as base_model should be a prediction of 0

In [68]:
submission = pd.DataFrame({'PassengerId':test.PassengerId,'Survived':0})

submission.to_csv('basemodeltitanic.csv',index = False)

In [69]:
cd

C:\Users\sidharth nandal


In [71]:
# dummify the data
newX = pd.get_dummies(X, drop_first=True)
newtest = pd.get_dummies(newtest,drop_first=True)

In [73]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
pred = lg.fit(newX,y).predict(newtest)

In [82]:
submission = pd.DataFrame({'PassengerId':test.PassengerId,'Survived':pred})

submission.to_csv('logistic_titanic.csv',index = False)   # 76.35

In [83]:
cd

C:\Users\sidharth nandal


### Model metrics and evaluation
* VIF 
* Logit Model
* Interpretation of coefficients
* Prediction using sigmoid
* psuedo R2
* Deviance 
* AIC

In [84]:
import statsmodels.api as sma

model = sma.Logit(y, newX).fit()
model.summary()

         Current function value: 0.397365
         Iterations: 35


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      873
Method:                           MLE   Df Model:                           17
Date:                Sun, 10 Dec 2023   Pseudo R-squ.:                  0.4033
Time:                        00:46:31   Log-Likelihood:                -354.05
converged:                      False   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 6.198e-91
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Pclass               -0.8026      0.187     -4.292      0.000      -1.169      -0.436
Age                  -0.0234      0.009     -2.505      0.012      -0.042      -0.005
SibSp               -27.5495   1.15e+05     -0.000      1.000   -2.25e+05    2.25e+05
Parch               -27.4419   1.15e+05     -0.000      1.000   -2.25e+05    2.25e+05
Fare                  0.0034      0.003      1.314      0.189      -0.002       0.009
Family               27.4698   1.15e+05      0.000      1.000   -2.25e+05    2.25e+05
Sex_male            -25.2262   1.15e+05     -0.000      1.000   -2.25e+05    2.25e+05
Embarked_Q            0.0330      0.401      0.082      0.935      -0.754       0.819
Embarked_S           -0.3189      0.254     -1.256      0.209      -0.816       0.179
cabin_cat_missing    -0.6404      0.322     -1.991      0.047      -1.271      -0.010
Titles_Miss         -25.8918   1.15e+05     -0.000      1.000   -2.25e+05    2.25e+05
Titles_Mr            -3.5874      0.598     -6.003      0.000      -4.759      -2.416
Titles_Mrs          -25.2120   1.15e+05     -0.000      1.000   -2.25e+05    2.25e+05
Titles_Ms           -11.6452   1.15e+05     -0.000      1.000   -2.25e+05    2.25e+05
Titles_others        -3.5582      0.824     -4.316      0.000      -5.174      -1.942
family_cat_duo        2.5015      1.291      1.937      0.053      -0.029       5.032
family_cat_small      2.6466      0.974      2.717      0.007       0.737       4.556
family_cat_solo       2.8819      1.536      1.877      0.061      -0.128       5.892
=====================================================================================
"""

### Basic Inference on the output
* There are lot of features which have a very high pvalue which indidcates the feature are statistically insignificant

* This also indicates that there is lot of multicolinearity between the categorical feature

* warning also suggest that the model os not a good model because ML;E could not help converge the sigmoid curve

In [90]:
# Lets apply VIF on newx
# note- cramerv to find corelation between categorical features

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = []

for i in range(newX.shape[1]):
    vif.append(variance_inflation_factor(newX.values, i))
    
pd.DataFrame(vif, columns = ['Value'], index = newX.columns).sort_values(ascending = False, by ='Value')

,Value
Family,2456.421885
SibSp,659.557404
Parch,367.754852
Sex_male,45.570269
family_cat_solo,40.915948
Titles_Miss,38.104557
Titles_Mrs,29.624656
family_cat_duo,19.136682
family_cat_small,9.467517
Titles_Mr,8.477798


In [91]:
# dropping some columns to remove the multicolinearity

subset = newX.drop(['SibSp','Parch','Titles_Mr'], axis = 1)

In [92]:
# Lets rebuild the model
model = sma.Logit(y, subset).fit()
model.summary()

         Current function value: 0.423569
         Iterations: 35


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      876
Method:                           MLE   Df Model:                           14
Date:                Sun, 10 Dec 2023   Pseudo R-squ.:                  0.3639
Time:                        01:20:44   Log-Likelihood:                -377.40
converged:                      False   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 2.424e-83
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Pclass               -0.7468      0.177     -4.226      0.000      -1.093      -0.400
Age                  -0.0419      0.009     -4.747      0.000      -0.059      -0.025
Fare                  0.0028      0.002      1.159      0.246      -0.002       0.007
Family                0.2379      0.157      1.513      0.130      -0.070       0.546
Sex_male             -1.7152      1.040     -1.650      0.099      -3.753       0.322
Embarked_Q            0.1628      0.390      0.417      0.676      -0.602       0.927
Embarked_S           -0.2965      0.244     -1.213      0.225      -0.776       0.183
cabin_cat_missing    -0.7675      0.309     -2.484      0.013      -1.373      -0.162
Titles_Miss           0.7279      1.059      0.687      0.492      -1.348       2.804
Titles_Mrs            1.5553      1.085      1.433      0.152      -0.572       3.683
Titles_Ms            21.2730   4.64e+04      0.000      1.000   -9.09e+04     9.1e+04
Titles_others         0.2715      0.538      0.505      0.614      -0.782       1.325
family_cat_duo        3.1949      0.788      4.056      0.000       1.651       4.739
family_cat_small      3.4625      0.659      5.255      0.000       2.171       4.754
family_cat_solo       3.6750      0.917      4.006      0.000       1.877       5.473
=====================================================================================
"""

### Lets remove the high pvalue feature

In [97]:
local_df = pd.DataFrame(model.pvalues, columns = ['pvalues']).reset_index()

In [139]:
feats = list(local_df.loc[local_df['pvalues']<0.05,'index'])

In [140]:
feats = ['Pclass','Fare',
 'Age','Family',
 'cabin_cat_missing',
 'family_cat_duo',
 'family_cat_small',
 'family_cat_solo']

In [138]:
# model of important features
newfeats = subset.loc[:, feats]
newfeats

,Pclass,Fare,Age,Family,cabin_cat_missing,family_cat_duo,family_cat_small,family_cat_solo
0,3,7.2500,22.0,2,1,1,0,0
1,1,71.2833,38.0,2,0,1,0,0
2,3,7.9250,26.0,1,1,0,0,1
3,1,53.1000,35.0,2,0,1,0,0
4,3,8.0500,35.0,1,1,0,0,1
...,...,...,...,...,...,...,...,...
886,2,13.0000,27.0,1,1,0,0,1
887,1,30.0000,19.0,1,0,0,0,1
888,3,23.4500,29.0,4,1,0,1,0
889,1,30.0000,26.0,1,0,0,0,1


In [137]:
model = sma.Logit(y, newfeats).fit()
model.summary()
# Your model is a better model now

Optimization terminated successfully.
         Current function value: 0.557573
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      883
Method:                           MLE   Df Model:                            7
Date:                Sun, 10 Dec 2023   Pseudo R-squ.:                  0.1627
Time:                        02:24:39   Log-Likelihood:                -496.80
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 3.381e-38
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Pclass               -0.5763      0.142     -4.052      0.000      -0.855      -0.298
Fare                  0.0049      0.002      2.139      0.032       0.000       0.009
Age                  -0.0366      0.007     -5.194      0.000      -0.050      -0.023
Family                0.1039      0.086      1.208      0.227      -0.065       0.272
cabin_cat_missing    -0.6823      0.255     -2.680      0.007      -1.181      -0.183
family_cat_duo        2.4748      0.360      6.875      0.000       1.769       3.180
family_cat_small      2.6633      0.343      7.766      0.000       1.991       3.335
family_cat_solo       1.9811      0.382      5.189      0.000       1.233       2.729
=====================================================================================
"""

### Predict the output using Sigmoid manually for our understanding

In [133]:
model.params

Pclass              -0.576341
Fare                 0.004883
Age                 -0.036571
Family               0.103851
cabin_cat_missing   -0.682343
family_cat_duo       2.474785
family_cat_small     2.663275
family_cat_solo      1.981147
dtype: float64

In [134]:
# get the all the columns from newtest mentioned in our list feats
newtest.loc[:,feats].head(1)

,Pclass,Fare,Age,Family,cabin_cat_missing,family_cat_duo,family_cat_small,family_cat_solo
891,3,7.8292,34.5,1,1,0,0,1


### Equation for prediction

* log(odds) = beta1* fare + beta2* family + beta3*cabin_cat_missing+.....

In [141]:
logit = model.params[0]*newtest['Pclass']+model.params[1]*newtest['Fare']+\
model.params[2]*newtest['Age']+model.params[3]*newtest['Family']+\
model.params[4]*newtest['cabin_cat_missing']+model.params[5]*newtest['family_cat_duo']+\
model.params[6]*newtest['family_cat_small']+model.params[7]*newtest['family_cat_solo']

In [150]:
# 1/(1+np.exp(-logit)) 0.175112
prob = 1/(1+np.exp(-logit)) 
print('prob using sigmoid', pd.DataFrame(prob).head(1))
direct = model.predict(newtest.loc[:,feats].head(1))
print('direct function',direct) 

prob using sigmoid             0
891  0.175112
direct function 891    0.175112
dtype: float64


- here we matched the value that we got by manually predicting with the values predicted by using the function

### Interpretations of Coefficients

In [151]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      883
Method:                           MLE   Df Model:                            7
Date:                Sun, 10 Dec 2023   Pseudo R-squ.:                  0.1627
Time:                        02:47:55   Log-Likelihood:                -496.80
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 3.381e-38
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Pclass               -0.5763      0.142     -4.052      0.000      -0.855      -0.298
Fare                  0.0049      0.002      2.139      0.032       0.000       0.009
Age                  -0.0366      0.007     -5.194      0.000      -0.050      -0.023
Family                0.1039      0.086      1.208      0.227      -0.065       0.272
cabin_cat_missing    -0.6823      0.255     -2.680      0.007      -1.181      -0.183
family_cat_duo        2.4748      0.360      6.875      0.000       1.769       3.180
family_cat_small      2.6633      0.343      7.766      0.000       1.991       3.335
family_cat_solo       1.9811      0.382      5.189      0.000       1.233       2.729
=====================================================================================
"""

In [152]:
np.exp(0.0049) # increase of odds

# By an increase in Fare by 1$ the odd of surving increased by 100x

1.0049120246322103

### Pseudo R2 

* **McFadden** - Here R2 is not similar to the Linear Regression. There in we interpret the R2 as the ratio of the variance in Y explained by X

* Because the variance in Y is not explained by X in logistic regression, however R2 exists and therefore it is called Pseudo R2

* Therefore McFadden R2 is calculated by taking log of your likelihood of your full model devided by your likelihood of your null model(LL_full/LL_null). This is (1 - LLF/LLN)

* Note- the value of McFadden R2 will be in a range of 0 and 1
* here **0 represents that the model has no explanatory power whereas 1 represent that the model has very high explanatory power**

* The reange of McFadde from 0.2-0.4 , genrally if the model has the R2 fallin in this range than we can say that the model is a good model

In [155]:
# McFadden
mcFadden = 1-(model.llf/model.llnull)
print('McFadden R2',mcFadden)

McFadden R2 0.16269210851308646


In [158]:
# Cox and snell

lo = np.exp(model.llnull)
l1 = np.exp(model.llf) 

cox_snell = 1- (lo/l1) ** (2/newX.shape[0])

In [160]:
# Nagelkerke

nagelkerke = cox_snell/(1-lo**(2/newX.shape[0]))

print('McFadden', mcFadden)
print('cox and snell',cox_snell)
print('nagelkerke',nagelkerke)

McFadden 0.16269210851308646
cox and snell 0.1948101945094688
nagelkerke 0.2646860770499358


### Which one to go with?

* McFadden is generally a conservative estimate
* Cox and snell is used to compare the nested models and thus it arries a lot of referenece there, because the value R2 is adjusted on the basis of n.

* Nagelkereke is based on cox and snell but it is more better estimate because it can easily achieve vales closer to 1 ( not really) but it is the higest R2 amonst all

* therefore is a goto R2 value for the model